In [8]:
#Import dependencies

import pandas as pd
import numpy as np
import glob
import datetime as dt
from datetime import timedelta
import time
import schedule
import csv
import xlsxwriter
import xlwt
from openpyxl.workbook import Workbook
from openpyxl.worksheet.properties import WorksheetProperties, PageSetupProperties
from pandas import ExcelWriter
from win32com.client import Dispatch
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 500)
from selenium.webdriver.common.action_chains import ActionChains
import shutil
from bs4 import BeautifulSoup
from config import password
import requests

In [106]:
options = webdriver.ChromeOptions()
profile = {
        'download.default_directory' : 'C:/Users/dshaf/Documents/Fantasy Baseball/',
        'download.prompt_for_download' : False,
        'download.extensions_to_open' : ''
}

options.add_experimental_option('prefs', profile)
browser = webdriver.Chrome(executable_path=r'C:/Users/dshaf/Documents/Fantasy Baseball/chromedriver.exe', options = options)
espn_url = 'http://www.espn.com/login'

df_agg = pd.DataFrame(columns=['Week','Team','Opponent','H','R','HR','TB','RBI','BB','K','SB','AVG','OPS',\
                               'IP','HRA','QS','W','L','SV','BAA','ERA','WHIP','K/9','Cat Wins','Cat Losses','Cat Ties'])

browser.get(espn_url)
time.sleep(5)
iframe = browser.find_element_by_id('disneyid-iframe')
browser.switch_to.frame(iframe)
browser.find_element_by_xpath("//input[@type='email']").send_keys('dshaffer8641@gmail.com')
browser.find_element_by_xpath("//input[@type='password']").send_keys(password)
time.sleep(3)
browser.find_element_by_class_name('btn').click()
time.sleep(10)
browser.find_element_by_link_text('Fantasy').click()
time.sleep(3)
browser.find_element_by_xpath('//span[text()="Bryzzo Souvenir Co."]').click()
time.sleep(5)
browser.find_element_by_xpath('//span[text()="Scoreboard"]').click()
time.sleep(7)



for w in range(1,6):
    
    browser.find_element_by_class_name('dropdown__select').click()
    time.sleep(2)
    browser.find_element_by_xpath(f'//option[@value="{w}"]').click()
    time.sleep(10)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    td = soup.select('td')
    
    lst = []

    for txt in np.arange(0, len(td)):
        if not td[txt].text.startswith('BattingPitching'):
            lst.append(td[txt].text)
    
    df = pd.DataFrame(columns=['Week','Team','Opponent','H','R','HR','TB','RBI','BB','K','SB','AVG','OPS',\
                               'IP','HRA','QS','W','L','SV','BAA','ERA','WHIP','K/9','Cat Wins','Cat Losses','Cat Ties'])
    
    #Create the dataframe using the parsed text
    df['Team'] = lst[0::21]
    df['H'] = lst[1::21]
    df['R'] = lst[2::21]
    df['HR'] = lst[3::21]
    df['TB'] = lst[4::21]
    df['RBI'] = lst[5::21]
    df['BB'] = lst[6::21]
    df['K'] = lst[7::21]
    df['SB'] = lst[8::21]
    df['AVG'] = lst[9::21]
    df['OPS'] = lst[10::21]
    df['IP'] = lst[11::21]
    df['HRA'] = lst[12::21]
    df['QS'] = lst[13::21]
    df['W'] = lst[14::21]
    df['L'] = lst[15::21]
    df['SV'] = lst[16::21]
    df['BAA'] = lst[17::21]
    df['ERA'] = lst[18::21]
    df['WHIP'] = lst[19::21]
    df['K/9'] = lst[20::21]
    df['Week'] = f'{w}'


    #Set the opponent column values
    df.loc[0]['Opponent'] = df.loc[1]['Team']
    df.loc[1]['Opponent'] = df.loc[0]['Team']
    df.loc[2]['Opponent'] = df.loc[3]['Team']
    df.loc[3]['Opponent'] = df.loc[2]['Team']
    df.loc[4]['Opponent'] = df.loc[5]['Team']
    df.loc[5]['Opponent'] = df.loc[4]['Team']
    df.loc[6]['Opponent'] = df.loc[7]['Team']
    df.loc[7]['Opponent'] = df.loc[6]['Team']
    df.loc[8]['Opponent'] = df.loc[9]['Team']
    df.loc[9]['Opponent'] = df.loc[8]['Team']

    #Set the category wins, losses, and ties values
    divs = soup.findAll('div', {'class': 'ScoreCell__Score h4 clr-gray-01 fw-heavy tar ScoreCell_Score--scoreboard pl2'})
    wins = []
    losses = []
    ties = []

    for div in divs:
      wins.append(div.text.split('-')[0])

    for div in divs:
      losses.append(div.text.split('-')[1])

    for div in divs:
      ties.append(div.text.split('-')[2])

    df['Cat Wins'] = wins
    df['Cat Losses'] = losses
    df['Cat Ties'] = ties
    
    df_agg = df_agg.append(df)

display(df_agg)

,Week,Team,Opponent,H,R,HR,TB,RBI,BB,K,SB,AVG,OPS,IP,HRA,QS,W,L,SV,BAA,ERA,WHIP,K/9,Cat Wins,Cat Losses,Cat Ties
0,1,PGGS,BRYZ,93,48,22,175,62,45,91,6,.231,.749,81.0,5,6,9,2,5,.164,1.33,0.81,10.11,11,9,0
1,1,BRYZ,PGGS,112,69,21,201,66,62,112,4,.253,.807,66.0,10,2,3,5,12,.285,4.64,1.44,10.64,9,11,0
2,1,C187,FSU!,121,72,15,189,57,41,92,2,.272,.767,73.0,15,3,3,4,6,.281,4.32,1.53,11.34,9,11,0
3,1,FSU!,C187,105,70,21,192,56,61,125,7,.248,.800,92.2,16,10,4,8,0,.215,3.50,1.19,9.32,11,9,0
4,1,.com,BYE,106,59,22,200,77,54,101,9,.256,.832,77.0,9,4,4,2,5,.235,4.09,1.30,9.70,12,8,0
5,1,BYE,.com,96,43,12,162,50,46,90,4,.251,.759,79.0,10,7,5,4,2,.166,3.30,0.99,11.85,8,12,0
6,1,OD,CAP,103,50,14,175,49,47,97,11,.248,.748,75.2,9,5,5,5,2,.236,3.45,1.27,8.56,9,11,0
7,1,CAP,OD,110,76,17,181,57,45,98,16,.274,.804,73.2,13,3,3,8,5,.233,4.76,1.30,9.16,11,9,0
8,1,DRDN,Jake,101,67,18,177,58,50,92,4,.262,.807,86.1,11,5,5,8,7,.244,4.38,1.31,10.11,13,5,2
9,1,Jake,DRDN,81,47,9,122,37,42,93,9,.223,.647,74.1,11,3,4,3,7,.211,3.75,0.97,7.99,5,13,2


In [ ]:
##### NOT PART OF THE PROGRAM #####

cols = ['Team','H','R','HR','TB','RBI','BB','K','SB','AVG','OPS',\
                            'IP','HRA','QS','W','L','SV','BAA','ERA','WHIP','K/9']
for i,j in enumerate(cols):

In [13]:
print(soup.prettify())

<html>
 <head>
  <meta charset="utf-8" class="next-head next-head"/>
  <title class="next-head">
   League Scoreboard - The Republic - ESPN Fantasy Baseball
  </title>
  <meta content="Play ESPN Fantasy Baseball" name="description"/>
  <meta content="116656161708917" property="fb:app_id"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="Play ESPN Fantasy Baseball" property="og:description"/>
  <meta content="http://g.espncdn.com/s/flblm/share.png" property="og:image"/>
  <meta content="630" property="og:image:height"/>
  <meta content="1200" property="og:image:width"/>
  <meta content="ESPN.com" property="og:site_name"/>
  <meta content="Fantasy Baseball Team Clubhouse - ESPN" property="og:title"/>
  <meta content="article" property="og:type"/>
  <meta content="http://fantasy.espn.com/baseball/team?leagueId=46514&amp;teamId=1&amp;seasonId=2019" property="og:url"/>
  <meta content="summary_large_image" name="twitter:card"/>
  <meta content="Play ESPN F

In [109]:
df_agg.sort_values(['ERA'], ascending=True).reset_index(drop=True)

,Week,Team,Opponent,H,R,HR,TB,RBI,BB,K,SB,AVG,OPS,IP,HRA,QS,W,L,SV,BAA,ERA,WHIP,K/9,Cat Wins,Cat Losses,Cat Ties
0,1,PGGS,BRYZ,93,48,22,175,62,45,91,6,.231,.749,81.0,5,6,9,2,5,.164,1.33,0.81,10.11,11,9,0
1,3,BRYZ,Jake,64,40,17,125,42,29,65,6,.259,.856,54.1,8,5,7,1,8,.168,1.82,0.74,11.93,9,9,2
2,3,.com,C187,75,44,15,145,41,35,62,2,.265,.865,47.2,5,5,5,1,1,.199,2.08,1.03,10.20,13,6,1
3,2,DRDN,BYE,65,38,14,126,46,36,57,8,.258,.857,57.1,4,6,5,1,4,.169,2.20,0.77,11.62,15,5,0
4,5,FSU!,BRYZ,87,54,14,148,51,28,65,3,.290,.844,53.1,7,6,7,4,1,.197,2.53,0.99,9.79,14,5,1
5,5,C187,DRDN,68,38,16,133,42,21,56,1,.269,.855,59.1,5,5,4,2,2,.236,2.88,1.13,8.65,15,4,1
6,5,CAP,.com,60,29,10,105,26,18,58,3,.225,.671,40.1,5,3,2,1,7,.260,2.90,1.36,9.60,5,14,1
7,2,OD,BRYZ,79,41,15,147,46,21,82,9,.266,.818,57.0,3,5,6,2,4,.199,3.00,1.02,8.84,15,5,0
8,5,OD,Jake,70,49,11,133,37,34,64,9,.245,.790,43.1,4,3,2,2,5,.221,3.12,1.11,10.38,12,8,0
9,3,C187,.com,74,39,15,141,43,29,54,4,.278,.890,45.1,6,4,3,4,4,.221,3.18,1.17,7.35,6,13,1
